In [10]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import functools as ft

In [11]:
url = 'https://app.youneedabudget.com/46c718b9-f242-4d73-8541-440fa0c82a31/budget/202205'

In [12]:
request = requests.get(url)
response = request.json
response
soup = BeautifulSoup(request.content,"html.parser")
mysoup = soup.html

NameError: name 'BeautifulSoup' is not defined

In [7]:
from __future__ import print_function
import time
import ynab
from ynab.rest import ApiException
from pprint import pprint

configuration = ynab.Configuration()

configuration.api_key['Authorization'] = '64Edz_HUqtPeGZpqSoQBDdwzoloAxY_wkHr-xJwjwo8'

configuration.api_key_prefix['Authorization'] = 'Bearer'

api_instance = ynab.AccountsApi()
api_categories = ynab.CategoriesApi(ynab.ApiClient(configuration))

budget_id = '46c718b9-f242-4d73-8541-440fa0c82a31'
account_id = '6db08996-92d2-446f-8902-e377e376f4d0'
account_id2 = "c74cee68-4f6f-4ed9-b520-367a9dbf33fe"
account_id3 = "538ffe77-426a-4d3d-bbde-8295013ea034"

try:
    account_data = api_instance.get_account_by_id(budget_id,account_id)
    account_data2 = api_instance.get_account_by_id(budget_id,account_id2)
    account_data3 = api_instance.get_account_by_id(budget_id,account_id3)
    account_categories = api_categories.get_categories(budget_id)
    
except ApiException as Exc:
    print("Exception when calling AccountsApi->get_account_by_id: %s\n" % Exc)

Exception when calling AccountsApi->get_account_by_id: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Server': 'Cowboy', 'Date': 'Sat, 17 Sep 2022 18:36:14 GMT', 'Connection': 'keep-alive', 'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding, Origin', 'Cache-Control': 'no-cache', 'X-Request-Id': 'f5629483-e7d9-40ca-b518-eff1fd15125a', 'X-Runtime': '0.034453', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains', 'Transfer-Encoding': 'chunked', 'Via': '1.1 vegur'})
HTTP response body: {"error":{"id":"403.2","name":"trial_expired","detail":"The trial for this account has expired."}}




In [2]:
account_data

NameError: name 'account_data' is not defined

In [592]:
json_string = json.loads(json.dumps(account_data.to_dict()))
df_data = pd.json_normalize(json_string)
df_data_clean = df_data[{"data.account.name","data.account.type"
                         ,"data.account.balance","data.account.cleared_balance","data.account.uncleared_balance"}]
df_data_clean.set_index("data.account.name")

,data.account.type,data.account.uncleared_balance,data.account.balance,data.account.cleared_balance
data.account.name,,,,
DtavoCash,savings,0.0,5000000.0,5000000.0


In [593]:
json_string2 = json.loads(json.dumps(account_data2.to_dict()))
df_data2 = pd.json_normalize(json_string2)
df_data_clean2 = df_data2[{"data.account.name","data.account.type"
                         ,"data.account.balance","data.account.cleared_balance","data.account.uncleared_balance"}]
df_data_clean2.set_index("data.account.name")

,data.account.type,data.account.uncleared_balance,data.account.balance,data.account.cleared_balance
data.account.name,,,,
DtavoCreditCard,creditCard,0.0,-20000000.0,-20000000.0


In [594]:
json_string3 = json.loads(json.dumps(account_data3.to_dict()))
df_data3 = pd.json_normalize(json_string3)
df_data_clean3 = df_data3[{"data.account.name","data.account.type"
                         ,"data.account.balance","data.account.cleared_balance","data.account.uncleared_balance"}]
df_data_clean3.set_index("data.account.name")

,data.account.type,data.account.uncleared_balance,data.account.balance,data.account.cleared_balance
data.account.name,,,,
DtavoChecking,checking,0.0,30000000.0,30000000.0


In [595]:
helper = df_data_clean.append(df_data_clean2, ignore_index=True)
Final_df = helper.append(df_data_clean3, ignore_index=True)
Final_df['data.account.uncleared_balance'] = 0.001 * Final_df['data.account.uncleared_balance']
Final_df['data.account.balance'] = 0.001 * Final_df['data.account.balance']
Final_df['data.account.cleared_balance'] = 0.001 * Final_df['data.account.cleared_balance']
Final_df = Final_df.set_index("data.account.name")
Final_df

,data.account.type,data.account.uncleared_balance,data.account.balance,data.account.cleared_balance
data.account.name,,,,
DtavoCash,savings,0.0,5000.0,5000.0
DtavoCreditCard,creditCard,0.0,-20000.0,-20000.0
DtavoChecking,checking,0.0,30000.0,30000.0


In [596]:
Final_df= Final_df.append(Final_df.sum(numeric_only=True).rename('Total'))
Final_df = Final_df.fillna('')

In [597]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat2 = pd.json_normalize(json_string["data"]["category_groups"][1]["categories"])
df_cat2 = df_cat2[{"name","budgeted","balance"}]
df_cat2

,name,balance,budgeted
0,DtavoCreditCard,4000000.0,4000000.0


In [598]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat3 = pd.json_normalize(json_string["data"]["category_groups"][2]["categories"])
df_cat3 = df_cat3[{"name","budgeted","balance"}]
df_cat3

,name,balance,budgeted
0,Car,5000000.0,5000000.0
1,Rent/Mortgage,5300000.0,5300000.0
2,Electric,100000.0,100000.0
3,Water,150000.0,150000.0
4,Internet,450000.0,450000.0
5,Cellphone,500000.0,500000.0


In [599]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat4 = pd.json_normalize(json_string["data"]["category_groups"][3]["categories"])
df_cat4 = df_cat4[{"name","budgeted","balance"}]
df_cat4 = df_cat4[:2]
df_cat4

,name,balance,budgeted
0,Groceries,5000000.0,5000000.0
1,Eating Out,2000000.0,2000000.0


In [600]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat5 = pd.json_normalize(json_string["data"]["category_groups"][4]["categories"])
df_cat5 = df_cat5[{"name","budgeted","balance"}]
df_cat5 = df_cat5[:1]
df_cat5

,name,balance,budgeted
0,Home Maintenance,100000.0,100000.0


In [601]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat6 = pd.json_normalize(json_string["data"]["category_groups"][5]["categories"])
df_cat6 = df_cat6[{"name","budgeted","balance"}]
df_cat6 = df_cat6[:1]
df_cat6

,name,balance,budgeted
0,Vacation,5000000.0,5000000.0


In [602]:
json_string = json.loads(json.dumps(account_categories.to_dict()))
df_cat7 = pd.json_normalize(json_string["data"]["category_groups"][6]["categories"])
df_cat7 = df_cat7[{"name","budgeted","balance"}]
df_cat7

,name,balance,budgeted
0,Hobbies,2500000.0,2500000.0
1,Entertainment,189000.0,189000.0
2,Health & Wellness,399000.0,399000.0


In [603]:
df_final = df_cat2.append(df_cat3).append(df_cat4).append(df_cat5).append(df_cat6).append(df_cat7)
df_final["balance"] = 0.001 * df_final["balance"] 
df_final["budgeted"] = 0.001 * df_final["budgeted"] 
df_final.reset_index()
df_final = df_final.set_index('name')
df_final= df_final.append(df_final.sum(numeric_only=True).rename('Total'))

In [604]:
df_final

,balance,budgeted
name,,
DtavoCreditCard,4000.0,4000.0
Car,5000.0,5000.0
Rent/Mortgage,5300.0,5300.0
Electric,100.0,100.0
Water,150.0,150.0
Internet,450.0,450.0
Cellphone,500.0,500.0
Groceries,5000.0,5000.0
Eating Out,2000.0,2000.0


In [605]:
Final_df

,data.account.type,data.account.uncleared_balance,data.account.balance,data.account.cleared_balance
data.account.name,,,,
DtavoCash,savings,0.0,5000.0,5000.0
DtavoCreditCard,creditCard,0.0,-20000.0,-20000.0
DtavoChecking,checking,0.0,30000.0,30000.0
Total,,0.0,15000.0,15000.0
